# RNA Velocity

As a powerful assessment of cell status, RNA velocity analyses expression dynamics based on scRNA-seq data, which generates the spliced and unspliced matrices in loom file, also called RNA abundance.

For example, we use exon information of GEM/GEF file to generate the spliced and unspliced matrices in the loom file. Therefore, it is necessary to ensure that GEM/GEF file is obtained from `spatial_RNA_visualization_v5` in SAP or SAW (version >= 5.1.3). 

The annotation logic in SAW is calculating overlap area with exon. If the overlap area with exon information is greater than 50%, it is considered that the transcript belongs to exon.

## Loom File

Import the module to generate loom file, please download our [example data]((https://pan.genomics.cn/ucdisk/s/BvIrye)) previously.

In [ ]:
from stereo.tools import generate_loom

bgef_file = './SS200000135TL_D1.tissue.gef'
gtf_file = './genes.gtf'
out_dir = './SS200000135TL_D1_bgef'

# generate loom file
loom_data = generate_loom(
                gef_path=bgef_file, 
                gtf_path=gtf_file, 
                bin_type='bins', 
                bin_size=100, 
                out_dir=out_dir
                )

<div class="alert alert-info">

**Note**

If you want to generate loom file through GEM file, use the parameter `gem_path`. Pay attention to the parameter `bin_type` which must be set to `'cell_bins'` when use files of `.cellbin.gef` or `.cellbin.gem`.

</div>

## Analysis on Dynamo

We recommend Dynamo to subsequent analysis of RNA velocity. Now that Dynamo has been released to PyPI, learn about the usage on its [document](https://dynamo-release.readthedocs.io/en/latest/ten_minutes_to_dynamo.html). Dynamo now has been released to PyPI, you can install the PyPI version via: `pip install dynamo-release` and `pip install cdlib`.

In [ ]:
#pip install dynamo-release
#pip install cdlib
import dynamo as dyn

# set visualization mode
dyn.configuration.set_figure_params('dynamo', background='white')

# read data
data = dyn.read_loom(loom_data)
dyn.pp.recipe_monocle(
            data,
            num_dim=30,
            keep_filtered_genes = True
            )

dyn.tl.dynamics(data, model='stochastic', cores=60)
dyn.tl.moments(data)
dyn.tl.reduceDimension(data)

The spatial coordinates of each cell are stored in the index of `adata.obs.index` which consists of coordinate x and coordiante y.

In [ ]:
# generate lists for coordinates of x, y 
data.obs['x'] = list(map(lambda x: float(x.split("_")[0]) ,list(data.obs.index)))
data.obs['y'] = list(map(lambda x: float(x.split("_")[1]) ,list(data.obs.index)))

data.obsm['spatial'] = data.obs[['x', 'y']].values.astype(float)
data.obsm['X_spatial'] = data.obs[['x', 'y']].values.astype(float)

Leiden clustering after RNA velovity.

In [ ]:

dyn.tl.cell_velocities(
            data, 
            method='fp', 
            basis='X_spatial', 
            enforce=True,  
            transition_genes = list(data.var_names[data.var.use_for_pca])
            )

dyn.tl.cell_wise_confidence(data)
dyn.tl.leiden(data, result_key='spatial_leiden_res')

Display the streamline plot of RNA velocity.

In [ ]:

dyn.pl.streamline_plot(
            data, 
            color = 'spatial_leiden_res', 
            basis='X_spatial',
            quiver_length=6, 
            quiver_size=6,  
            show_arrowed_spines=True,
            figsize=(6, 6)
            )